# Scrapy + Scraper API

In [78]:
pip install scrapy

Note: you may need to restart the kernel to use updated packages.


In [79]:
import scrapy
from urllib.parse import urlencode
from urllib.parse import urlparse
import json
from datetime import datetime
API_KEY = 'afd4c282f739aadfe2b28f623ddb2b17'

In [80]:
def get_url(url):
   payload = {'api_key': API_KEY, 'url': url, 'autoparse': 'true', 'country_code': 'fr'}
   proxy_url = 'http://api.scraperapi.com/?' + urlencode(payload)
   return proxy_url

def create_google_url(query, site=''):
   google_dict = {'q': query, 'num': 100, }
   if site:
       web = urlparse(site).netloc
       google_dict['as_sitesearch'] = web
       return 'http://www.google.com/search?' + urlencode(google_dict)
   return 'http://www.google.com/search?' + urlencode(google_dict)

class GoogleSpider(scrapy.Spider):
   name = 'google'
   #allowed_domains = ['api.scraperapi.com']
   #custom_settings = {'ROBOTSTXT_OBEY': False, 'LOG_LEVEL': 'INFO', 'CONCURRENT_REQUESTS_PER_DOMAIN': 5, 'RETRY_TIMES': 0}
   allowed_domains = ['www.google.com']
   custom_settings = {'ROBOTSTXT_OBEY': False, 'LOG_LEVEL': 'INFO', 'CONCURRENT_REQUESTS_PER_DOMAIN': 5, 'RETRY_TIMES': 0}

   def start_requests(self):
       queries = ['innovation'] 
       for query in queries:
           url = create_google_url(query)
           yield scrapy.Request(get_url(url), callback=self.parse, meta={'pos': 0})

   def parse(self, response):
       di = json.loads(response.text)
       pos = response.meta['pos']
       dt = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
       for result in di['organic_results']:
           title = result['title']
           snippet = result['snippet']
           link = result['link']
           item = {'title': title, 'snippet': snippet, 'link': link, 'position': pos, 'date': dt}
           pos += 1
           yield item
       next_page = di['pagination']['nextPageUrl']
       if next_page:
           yield print(scrapy.Request(get_url(next_page), callback=self.parse, meta={'pos': pos}).text)

In [81]:
if __name__ == '__main__':
    result = list(GoogleSpider().start_requests())

In [82]:
result

[<GET http://api.scraperapi.com/?api_key=afd4c282f739aadfe2b28f623ddb2b17&url=http%3A%2F%2Fwww.google.com%2Fsearch%3Fq%3Dinnovation%26num%3D100&autoparse=true&country_code=fr>]

In [83]:
link = str(result[0])[5:-1]
link

'http://api.scraperapi.com/?api_key=afd4c282f739aadfe2b28f623ddb2b17&url=http%3A%2F%2Fwww.google.com%2Fsearch%3Fq%3Dinnovation%26num%3D100&autoparse=true&country_code=fr'

In [84]:
# On importe la fonction 'get' (téléchargement) de 'requests' 
# Et la classe 'Selector' (Parsing) de 'scrapy'
from requests import get
from scrapy import Selector
# Lien de la page à scraper
url = link
response = get(url)
source = None # Le code source de la page 
if response.status_code == 200 :
    # Si la requete s'est bien passee
    source = response.text

In [85]:
source

'{"search_information":{"total_results":840,"time_taken_displayed":0,"query_displayed":"innovation"},"ads":[],"knowledge_graph":{"title":"","description":"","source":{"name":""},"related":[],"related_link":null,"social_media":[],"see_more_about":[{"title":"Recherches associées","link":"https://www.google.com/search?num=100&gl=FR&q=Innovation&stick=H4sIAAAAAAAAAONgFuLWT9c3NDKyTMkuM1NC5mgJZydb6Sfn5-bm51kVZ6aklidWFi9i5fLMy8svSyzJzM_bwcoIALUghDRBAAAA&sa=X&ved=2ahUKEwi38aeQgIXuAhVSL6wKHaMUDegQMSgAMHh6BAgUEAE"},{"title":"Voir d\'autres éléments (plus de\xa010)","link":"https://www.google.com/search?num=100&gl=FR&q=Innovation&stick=H4sIAAAAAAAAAONgFuLWT9c3NDKyTMkuM1NC5mgJZydb6Sfn5-bm51kVZ6aklidWFi9i5fLMy8svSyzJzM_bwcoIALUghDRBAAAA&sa=X&ved=2ahUKEwi38aeQgIXuAhVSL6wKHaMUDegQzTooATB4egQIFBAC"},{"title":"Créativité","link":"https://www.google.com/search?num=100&gl=FR&q=Cr%C3%A9ativit%C3%A9&stick=H4sIAAAAAAAAAONgFuLWT9c3NDKyTMkuM1Pi1M_VNzA0MkjL0-Jzzs_Nzc8LzkxJLU-sLF7EyuNcdHhlYklmWWbJ4ZU7WBkBixItvz

In [102]:
import re
s = source
pattern = re.compile("\"link\"[^,]+")
my_links = list(set([x for x in pattern.findall(s)]))
my_links
my_links = [i[8:] for i in my_links]
my_links = [i[:-1] for i in my_links]

mylinks=[]

for i in my_links:
    if "https://www.google" not in i:
        mylinks.append(i)

mylinks

['https://www.sciencesetavenir.fr/tag_defaut/innovation_5391/',
 'https://www.framatome.com/FR/businessnews-1495/innovation.html',
 'https://www.lemondeinformatique.fr/innovation-informatique-87.html',
 'https://www.nexity.fr/groupe/innovation',
 'https://www.valdoise.fr/667-innovation.htm',
 'https://www.enseignementsup-recherche.gouv.fr/',
 'https://www.sival-innovation.com/',
 'https://trendemic.net/definition-creativite-innovation-imagination-invention/',
 'https://www.maddyness.com/category/innovation/',
 'https://www.unhcr.org/fr-fr/innovation.html',
 'https://www.midilibre.fr/2021/01/04/lantana-ecosylva-paysage-cultive-linnovation-et-la-proximite-9290971.php',
 'http://www.cite-sciences.fr/archives/francais/ala_cite/expositions/observatoire-innovations/definition-innovation/definition-innovation-1.html',
 'https://www.zabala.fr/',
 'https://arcinnovation.fr/',
 'https://www.petite-entreprise.net/articles/J-accompagne-les-dirigeants/Definition-innovation-Qu-est-ce-que-c-est-Pourq

't'

':https://www.google.com/search?num=100&gl=FR&q=Innovation&stick=H4sIAAAAAAAAAONgFuLQz9U3KMuwLFCCs7SEs5Ot9JPzc3Pz86yKM1NSyxMrixexcnnm5eWXJZZk5uftYGUEANz69kA7AAAA&sa=X&ved=2ahUKEwiHzLjT0ITuAhUcB50JHaEKC4AQzTooATB1egQIKBAC},{title:Creativity,'

,0
0,"{search_information:{total_results:1760000000,..."
1,":null,social_media:[],see_more_about:[{title:P..."
2,:https://www.google.com/search?num=100&gl=FR&q...
3,:https://www.google.com/search?num=100&gl=FR&q...
4,:https://www.google.com/search?num=100&gl=FR&q...
5,:https://www.google.com/search?num=100&gl=FR&q...
6,:https://www.google.com/search?num=100&gl=FR&q...
7,:https://www.google.com/search?num=100&gl=FR&q...
8,:https://www.google.com/search?num=100&gl=FR&q...
9,":https://en.wikipedia.org/wiki/Innovation,date..."
